# Vishnu Sahasranamam: shivkumar.org

Attempt to create individual meanings of the 1000 names of Vishnu from http://www.shivkumar.org/music/vishnu-sahasranamam-meanings.htm. The file that we have is very good in terms of how the various names are explained based on the Bhashya of Shankara Bhagavadpada. 

Our aim is to get the namas and the commentary from the file using BeautifulSoup.

In [1]:
from bs4 import BeautifulSoup, Comment
import textwrap, re
import os
from io import open # 2 and 3 compatibility
import urllib

# Very useful function

def remove_attrs(soup, whitelist=tuple()):
    for tag in soup.findAll(True):
        for attr in [attr for attr in tag.attrs if attr not in whitelist]:
            del tag[attr]
    return soup

Download the file is it does not already exist.

In [2]:
FILE_NAME = r"Vishnu Sahasranamam-shivkumar.org.html"

if not os.path.exists(FILE_NAME):
    response = urllib2.urlopen('http://python.org/')
    html = response.read()
    f = open(FILE_NAME, 'r', encode="urf-8").write(html)
else:
    html = open(FILE_NAME, "r").read()

In [3]:
soup = BeautifulSoup(html, 'html.parser')

The following may not be necassary, but at some point I did not want to see the comments

In [4]:
comments = soup.findAll(text=lambda text:isinstance(text, Comment))
c = [comment.extract() for comment in comments]

We create some utility function to mark the 'Shloka' part of the page. This we do by ensuring that the first of all we remove all atrributes from the span part, and when the span has a style that is aligned in the ceter, we mark that in our soup `s` using a class tag

In [5]:
def process_span(p):
    s = remove_attrs(p)
    if p.get('style') == r'text-align:center':
        s['class'] = 'Shloka'
    return s

This is the main part of the extraction routine. We first create some variables we use. 

In [6]:
number_pat = re.compile(r'^([ ]?\(?(\d+)[\)\.]?)')
sahasranams = []
number = []
namas = []
NEW_PATTERN = u'NeWWeN'

def process_soup(soup):
    # Get all the paragraphs until we come to the first namam.
    print_all = 0
    st = ""
    para = soup.findAll('p')
    f = open('vishnu-shivkumar.txt', 'w',encoding='utf-8')
    for i, p in enumerate(para):
        ## The start of the commentary
        if p.span.text == r'(1) Visvam':
            print_all = 1
        if print_all:
            s = process_span(p)
            ## Can we catch the start of a name and commentary from others.
            if (s.b != None):
                my_str = textwrap.fill(s.b.text).replace(u'\xa0', ' ')
            elif (p.strong != None):
                my_str = textwrap.fill(p.strong.text).replace(u'\xa0', ' ')
            else:
                my_str = ""
            m = number_pat.search(my_str)
            if m:
                if my_str:
                    nama = my_str[len(m.group(0)):]
                    namas.append(nama.strip())
                    sahasranams.append(my_str)
                    number.append(m.group(2))
                    f.write(NEW_PATTERN)
                    st += NEW_PATTERN
            ## This is the end of the commentary
            if s.text.startswith('Sree Sarva'):
                print_all = 0
            if s.get('class', None) == 'Shloka':
                pass
            elif s != "":
                f.write("%s"%s.text)
                st += "%s"%s.text
    return st


In [7]:
commentary_str = process_soup(soup)

## Corrections

This document had some names (from position 780 to 804 seem to be repeated. We delete them.

In [8]:
START_NO = 779
END_NO = 803

del number[START_NO:END_NO]
del sahasranams[START_NO:END_NO]

commentary = commentary_str.split('NeWWeN')
del commentary[0]
del commentary[START_NO:END_NO]

In [9]:
## Even after this there are only 999 names. Checking what is missing

che = [i + 1 - int(number[i]) for i in range(999)]
miss = che.index(-1)

print(miss)

## And sure enough that is missing

954


And we supply that.

In [10]:
number.insert(miss, '955')
sahasranams.insert(miss, 'Satpatachara')
commentary.insert(miss, "955 Satpatachar - Walking in the path of righteousness. He follows the path which good men \
                  have chalked out (in his incarnations)")

In [11]:
def get_commentary(n):
    k = re.sub(number_pat, "", sahasranams[n])
    c = re.sub(number_pat, "", commentary[n])
    c = c[len(k):].strip()
    c = c.strip()
    if (c.find('-') == 0):
        c = c[1:]
    k = k.strip()
    return(k, c)

In [12]:
def print_one_full_line(n):
    print(get_commentary(n))

In [13]:
number[10]

'11'

In [14]:
print_one_full_line(456)

('Sookshmah', 'One\nwho is subtler than the subtlest. In Vedanta terminology subtlety indicates\npervasiveness. Therefore, the term means All-Pervasive. The Upanishad says the\nLord is All-Pervading, subtler than the subtlest (Sarvagatam susookshmam).')


We print in shloka by shloka. We get the shloka information from vishnu-krishnanda information

In [15]:
fn = open("shloka-number.txt", "r", encoding="utf-8")
number = []

for line in fn:
    s, n = line.split()
    number.append(int(n))
number.append(1001)

In [16]:
print(number)

[1, 10, 18, 25, 37, 46, 55, 64, 74, 85, 95, 104, 114, 123, 133, 141, 151, 163, 173, 181, 189, 198, 209, 218, 228, 236, 247, 256, 265, 275, 283, 290, 300, 308, 318, 327, 336, 346, 355, 363, 373, 384, 394, 405, 416, 426, 435, 445, 455, 465, 475, 486, 494, 503, 513, 521, 531, 538, 547, 558, 567, 574, 586, 596, 605, 615, 624, 633, 642, 651, 661, 671, 679, 690, 699, 708, 717, 725, 737, 747, 757, 765, 773, 782, 790, 800, 807, 817, 826, 835, 847, 857, 867, 876, 886, 896, 906, 915, 923, 932, 941, 950, 959, 967, 976, 985, 993, 1001]


In [17]:
def process_names_to_shlokas(k, f):
    f.write(u"<sloka id = '%d'>\n"%(k+1))
    f.write(u'<ol start = %s>\n' % (number[k]))
    for i in range(number[k], number[k + 1]):
        x, y = get_commentary(i - 1)
        f.write(u"<li>%s: %s</li>\n"%(x, textwrap.fill(y)))
    f.write(u'</ol>\n')
    f.write(u'</sloka>\n')


f = open('vishnu-shivakumar-shankara.xml', 'w', encoding='utf-8')
for i in range(len(number) - 1):
    process_names_to_shlokas(i, f)
f.close()    

In [18]:
len(sahasranams)

1000

In [19]:
sahasranams[998]

'999. Akshobhyah'

In [20]:
namas.index('Vanamaalee')

560

In [21]:
print(namas)

['Visvam', 'Vishnuh', 'Vashatkaarah', 'Bhoota-bhavya-bhavat-prabhuh', 'Bhoota-krit', 'Bhoota-bhrit', 'Bhaavah', 'Bhootaatmaa', 'Bhoota-bhaavanah', 'Poota-atmaa', 'Parama-atmaa', 'Muktaanaam paramaa gatih', 'Avyaah', 'Purushah', 'Saakshee', 'Kshetrajnah', 'Aksharah', 'Yogah', 'Yoga-vidaam netaa', 'Pradhaana-purusha-eesvarah', 'Naarasimha-vapuh', 'Sreemaan', 'Keshavah', 'Purushottamah', 'Sarvah', 'Sharvah', 'Sivah', 'Sthaanuh', 'Bhootaadih', 'Avyayah Nidhih', 'Sambhavah', 'Bhaavanah', 'Bhartaa', 'Prabhavah', 'Prabhuh', 'Eesvarah', 'Svayambhooh', 'Shambhuh', 'Aadityah', 'Pushkaraakshah', 'Mahaasvanah', 'Anaadi-nidhanah', 'Dhaataa', 'Vidhaataa', 'Dhaatur-uttamah', 'Aprameyah', 'Hrisheekesah', 'Padmanaabhah', 'Amaraprabhuh', 'Visvakarmaa', 'Manuh', 'Tvashtaa', 'Sthavishthah', 'Sthaviro Dhruvah', 'Agraahyah', 'Saasvatah', 'Krishnah -', 'Lohitaakshah', 'Pra-tarda-nah', 'Prabhootah', 'Tri-kakub-dhaama', 'Pavitram –', 'Param Mangalam', 'Eesaanah', 'Praanadah', 'Praanah', 'Jyeshthah', 'Sreshthah